# Web scraping 🏴‍☠️

##### Tu web scraper de confianza

In [11]:
import pandas as pd
import os
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

### Extracción de tablas 📊

Para completar los requisitos del proyecto, extraigo información mediante web scraping.

In [12]:
url = "https://es.wikipedia.org/wiki/Anexo:Municipios_de_Espa%C3%B1a_por_poblaci%C3%B3n"

In [13]:
driver = webdriver.Chrome()
driver.get(url)

In [14]:
# Obtener el HTML de la página con BeautifulSoup
html = requests.get(url).content
soup = BeautifulSoup(html, "html.parser")

# Busco la tabla por su clase
table = soup.find('table', class_='wikitable')

# Convierto la tabla a un df
population = pd.read_html(str(table))[0]

In [15]:
population

,N.º,Nombre,Población (1 de enero de 2022),Provincia,Comunidad autónoma
0,1,Madrid,3 286 662,Madrid,Comunidad de Madrid
1,2,Barcelona,1 636 193,Barcelona,Cataluña
2,3,Valencia,792 492,Valencia,Comunidad Valenciana
3,4,Sevilla,681 998,Sevilla,Andalucía
4,5,Zaragoza,673 010,Zaragoza,Aragón
5,6,Málaga,579 076,Málaga,Andalucía


In [16]:
population.to_csv('../data/population.csv', index=False)

### Conexión con Mongo

In [49]:
import pymongo
from pymongo import MongoClient

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["ETL_project"]

In [50]:
collection = db["data"]
data_dict = population.to_dict(orient="records")

# Inserta los registros en la colección
collection.insert_many(data_dict)

### Extracción de imágenes para futuros trabajos 🍋

Además, he añadido la extracción de imágenes en pinterest. Para esta ocasión, he elegido al personaje [Limoncio](https://www.youtube.com/watch?v=dmxhLEoJf-o), de *Adventure time*.

In [43]:
url = 'https://www.pinterest.es/search/pins/?rs=ac&len=2&q=lemongrab%20adventure%20time&eq=Lemongrab%20adve&etslf=3860'

driver = webdriver.Chrome()
driver.get(url)

scroll_count = 5  # Scroll hacia abajo

destination_folder = os.path.join("../img", "lemongrab_images")

# Crea la carpeta si no existe
os.makedirs(destination_folder, exist_ok=True)

for i in range(scroll_count):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Espero a que se carguen las imágenes.

    # Encuentra las etiquetas de imagen (img) en la página
    img_tags = driver.find_elements(By.CSS_SELECTOR, 'img')
    print(f'Número de etiquetas de imágenes encontradas: {len(img_tags)}')

    for index, img in enumerate(img_tags):
        '''
        Con enumerate es lo mismo que hacer:
        
        for list in lists:
            for e in list:

        '''
        try:
            src = img.get_attribute('src') #Para cada etiqueta de imagen, intenta coger el atributo "src" con la URL de la imagen
            if src: #verifica si tiene un valor nulo
                if src.startswith("data:image/"):
                    # omite la descarga de imágenes con este tipo de URL, porque Las URL que comienzan con "data:image/" suelen representar imágenes incrustadas en la página en lugar de imágenes a las que se pueda acceder mediante una solicitud HTTP
                    continue
                response = requests.get(src)
                if response.status_code == 200:
                    image_filename = os.path.join(destination_folder, f"lemongrab_{index}.jpg") #Se genera el nombre del archivo de imagen que se utilizará para guardar la imagen en la carpeta local
                    with open(image_filename, 'wb') as image_file: #Se abre el archivo en modo binario ('wb') para escribir la imagen descargada.
                        image_file.write(response.content)
        except Exception as e:
            print(f"An error occurred: {e}")

# Cierra el navegador
driver.quit()

Número de etiquetas de imágenes encontradas: 23
Número de etiquetas de imágenes encontradas: 24
Número de etiquetas de imágenes encontradas: 22
Número de etiquetas de imágenes encontradas: 24
Número de etiquetas de imágenes encontradas: 22
